In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from datetime import datetime
from random import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KDTree
from datetime import datetime
from pulp import *

In [2]:
df_history = pd.read_csv('./data/Demand_History.csv')
df = pd.read_csv('./data/exisiting_EV_infrastructure_2018.csv')


# Predict Demand for 2019 and 2020

In [109]:
df_pred = pd.read_csv('./data/df_history.csv')

In [110]:
df_pred = df_history
polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    LinearRegression(),
)
X =np.array([2012,2013,2014,2015,2016,2017,2018])
pred_2019 = []
pred_2020 = []
for row,cols in df_history.iterrows():
    y = []
    for num,(i) in enumerate(X):
        y.append(df_history[str(i)][row])  
    polynomial_regression.fit(X.reshape(-1, 1), y)
    pred_2019.append(float(polynomial_regression.predict(np.array([2019]).reshape(-1, 1))))
    pred_2020.append(float(polynomial_regression.predict(np.array([2020]).reshape(-1, 1))))
df_pred['2019'] = pred_2019
df_pred['2020'] = pred_2020

In [305]:
df_pred.to_csv('./data/df_pred_pol.csv', index=False)

# Helper DataFrames

# Demand to supply Distance DF

In [ ]:
supply = []
distance = []
demand = []
counter = 0
for i, row in df_closest_supply.iterrows():

    for j in range(100):
        supply.append(int(row.closest_supply_index[j]))
        distance.append(float(row.supply_distance[j]))
        demand.append(int(i))
        counter +=1
df_distance_demand_supply = pd.DataFrame({'Demand_Index': demand, 'Supply_Index':supply, 'Distance':distance})
supply = []
distance = []
demand = []
for i in range(4096):
    print(i)
    for j in range(100):
        demand.append(i)
        supply.append(j)
        distance.append(float(df_distance_demand_supply[(df_distance_demand_supply['Demand_Index'] == i) & (df_distance_demand_supply['Supply_Index'] == j)]['Distance']))
df_distance_demand_supply = pd.DataFrame({'Demand_Index': demand, 'Supply_Index':supply, 'Distance':distance})        
 df_distance_demand_supply.to_csv('./data/distance_demand_supply.csv', index = False)    

# Algorithm for Optimization Demand and Supply points

In [116]:
#Load Data

df_distance_demand_supply = pd.read_csv('./data/distance_demand_supply.csv')
df_pred = pd.read_csv('./data/df_pred_pol.csv')

In [117]:
#Add Rising Factor based on market research

for i, row in df_pred.iterrows():
    df_pred.loc[i,'2019'] = df_pred.loc[i,'2019']*1.1
    df_pred.loc[i,'2020']= df_pred.loc[i,'2020']*1.25
df_pred.reset_index()
df_pred

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,17.076836,22.676043
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,16.183002,21.195882
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,19.066242,26.601385
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,21.126277,29.341391
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,22.131313,30.952561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,7.097168,9.605028
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.554221,3.471328
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,4.023162,5.368988
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,8.636681,11.421589


In [118]:
#Linear Optimization 2019

timestamp = datetime.now()

model = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2019 = df_pred['2019']
df_dict = df.to_dict('records')

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')

y = LpVariable.dicts('Supply_SC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')

z = LpVariable.dicts('Supply_FC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')


#Optimization Function
model += (lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )
         +600*lpSum([y[i['supply_point_index']]-i['existing_num_SCS'] for i in df_dict] + [z[i['supply_point_index']]-i['existing_num_FCS'] *1.5 for i in df_dict]))

#Constraint Number of build Chargers must be positive
for i in df_dict:
    model+= lpSum(y[i['supply_point_index']]) >= 0
    model+= lpSum(z[i['supply_point_index']]) >= 0

#Constraint Number of already buld chargers is minimum
for i in df_dict:
    model+= lpSum(y[i['supply_point_index']]) >= i['existing_num_SCS']
    model+= lpSum(z[i['supply_point_index']]) >= i['existing_num_FCS']

#Constraint Total Number of Chargers cant exceed parking spaces
for i in df_dict:
    model += lpSum(y[i['supply_point_index']]+z[i['supply_point_index']]) <= df_dict[i['supply_point_index']]['total_parking_slots']

#Constraint Charger must satify set demand for the year

model += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) >= df_2019.sum()

#Constraint Sum of Supply Satisfied per Demandpoint j, cant exceed Demandpoint j Capacity
for i in range(100):
    model += lpSum([x[(j,i)] for j in range(4096)]) <= y[i]*200 + z[i]* 400  
    
#Constraint Sum of Demand  per partial demandpoint i matrix equals predicted demand per whole demand point j
for i in range(4096):
    model += lpSum([x[(i, j)] for j in range(100)]) == df_2019[i]    

model.solve()

print(datetime.now()-timestamp)

    

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sebastianbott/miniforge3/envs/kerasEnv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/95289a2d806c49cf95ccbbb1892266d3-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/95289a2d806c49cf95ccbbb1892266d3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4702 COLUMNS
At line 1235103 RHS
At line 1239801 BOUNDS
At line 1240002 ENDATA
Problem MODEL has 4697 rows, 409800 columns and 820200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.643e+06 - 0.85 seconds
Cgl0004I processed model has 4267 rows, 406800 columns (200 integer (0 of which binary)) and 813800 elements
Cbc0012I Integer solution of 2643524.7 found by DiveCoefficient after 0 iterations and 0 nodes (5.

In [119]:
#Extract and Safe results

name = []
value = []
for v in model.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2019 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2019.to_csv('./Output/Highscore_tests/demand_to_supply_2019.csv', index=False)
name = []
value = []
for v in model.variables():
    if 'Supply_SC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_SC_rework_2019 = pd.DataFrame({'Name':name, 'SC': value})
df_SC_rework_2019['Supply_Index']  = df_SC_rework_2019.Name.str.extract('(\d+)', expand=False).astype(int)
df_SC_rework_2019.sort_values(by=['Supply_Index'], inplace = True)
name = []
value = []
for v in model.variables():
    if 'Supply_FC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_FC_rework_2019 = pd.DataFrame({'Name':name, 'FC': value})
df_FC_rework_2019['Supply_Index']  = df_SC_rework_2019.Name.str.extract('(\d+)', expand=False).astype(int)
df_FC_rework_2019.sort_values(by=['Supply_Index'], inplace = True)
supply_id = [i for i in range(100)]
supply_charger_2019 = pd.DataFrame({'supply_point_index':supply_id, 'existing_num_SCS':df_SC_rework_2019['SC'],'existing_num_FCS':df_FC_rework_2019['FC'] })
supply_charger_2019.reset_index(inplace=True)
supply_charger_2019.to_csv('./Output/Highscore_tests/supply_charger_2019.csv')


In [120]:
#Linear Optimization 2020

timestamp = datetime.now()

model2020 = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2020 = df_pred['2020']
for i in range(100):
    supply_charger_2019.loc[supply_charger_2019['supply_point_index'] == i,'total_parking_slots'] = int(df.loc[df['supply_point_index'] == i,'total_parking_slots'])
df_dict = supply_charger_2019.reset_index()
df_dict['total_parking_slots'] = df['total_parking_slots']
df_dict = df_dict.to_dict('records')

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts('Supply_SC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')

z = LpVariable.dicts('Supply_FC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')



model2020 += (lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )
         +600*lpSum([y[i['supply_point_index']]-i['existing_num_SCS'] for i in df_dict] + [z[i['supply_point_index']]-i['existing_num_FCS'] *1.5 for i in df_dict]))

#Number of Built Chargers cant be less than zero
for i in df_dict:
    model2020+= lpSum(y[i['supply_point_index']]) >= 0
    model2020+= lpSum(z[i['supply_point_index']]) >= 0

#Constraint Number of already buld chargers is minimum
for i in df_dict:
    model2020+= lpSum(y[i['supply_point_index']]) >= i['existing_num_SCS']
    model2020+= lpSum(z[i['supply_point_index']]) >= i['existing_num_FCS']

#Constraint Total Number of Chargers cant exceed parking spaces
for i in df_dict:
    model2020 += lpSum(y[i['supply_point_index']]+z[i['supply_point_index']]) <= df_dict[i['supply_point_index']]['total_parking_slots']

#Constraint Charger must satify set demand for the year

model2020 += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) >= df_2020.sum()

  
#Constraint #1 Sum of Supply Satisfied per Demandpoint j, cant exceed Supplypoint i Capacity
for i in range(100):
    model2020 += lpSum([x[(j,i)] for j in range(4096)]) <= y[i]*200 + z[i]* 400   
    
#Constraint #2 Sum of Demand per partial demandpoint i matrix equals predicted demand per whole demand point i
for i in range(4096):
    model2020 += lpSum([x[(i, j)] for j in range(100)]) == df_2020[i]    

model2020.solve()

print(datetime.now()-timestamp)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sebastianbott/miniforge3/envs/kerasEnv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/6069188926cc4a4eb409094bcce5a50c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/6069188926cc4a4eb409094bcce5a50c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4702 COLUMNS
At line 1235103 RHS
At line 1239801 BOUNDS
At line 1240002 ENDATA
Problem MODEL has 4697 rows, 409800 columns and 820200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.6613e+06 - 0.91 seconds
Cgl0004I processed model has 4248 rows, 406762 columns (162 integer (0 of which binary)) and 813686 elements
Cbc0012I Integer solution of 3661617.8 found by DiveCoefficient after 0 iterations and 0 nodes (5

In [121]:
#Extract and Safe results

name = []
value = []
for v in model2020.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2020 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2020.to_csv('./Output/Highscore_tests/demand_to_supply_2020.csv', index=False)
name = []
value = []
for v in model2020.variables():
    if 'Supply_SC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_SC_rework_2020 = pd.DataFrame({'Name':name, 'SC': value})
df_SC_rework_2020['Supply_Index']  = df_SC_rework_2020.Name.str.extract('(\d+)', expand=False).astype(int)
df_SC_rework_2020.sort_values(by=['Supply_Index'], inplace = True)
name = []
value = []
for v in model2020.variables():
    if 'Supply_FC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_FC_rework_2020 = pd.DataFrame({'Name':name, 'FC': value})
df_FC_rework_2020['Supply_Index']  = df_FC_rework_2020.Name.str.extract('(\d+)', expand=False).astype(int)
df_FC_rework_2020.sort_values(by=['Supply_Index'], inplace = True)
supply_id = [i for i in range(100)]
supply_charger_2020 = pd.DataFrame({'supply_point_index':supply_id, 'existing_num_SCS':df_SC_rework_2020['SC'],'existing_num_FCS':df_FC_rework_2020['FC'] })
supply_charger_2020.reset_index(inplace = True)
supply_charger_2020.to_csv('./Output/Highscore_tests/supply_charger_2020.csv')



# Prepare Solution

In [122]:

df_supply_2019 = pd.read_csv('./Output/Highscore_tests/supply_charger_2019.csv')
df_supply_2020 = pd.read_csv('./Output/Highscore_tests/supply_charger_2020.csv')
df_demand_2019 = pd.read_csv('./Output/Highscore_tests/demand_to_supply_2019.csv')
df_demand_2020 = pd.read_csv('./Output/Highscore_tests/demand_to_supply_2020.csv')

df_demand_2019['Supply_Index'] = pd.to_numeric(df_demand_2019['Supply_Index'],downcast='integer')
df_demand_2020['Supply_Index'] = pd.to_numeric(df_demand_2020['Supply_Index'],downcast='integer')
df_solution_demand_2020 = pd.DataFrame({'year':2020,'data_type':'DS','demand_point_index':df_demand_2020['Demand_Index'],'supply_point_index':df_demand_2020['Supply_Index'],'value':df_demand_2020['Demand_Satisfied']})
df_solution_demand_2019 = pd.DataFrame({'year':2019,'data_type':'DS','demand_point_index':df_demand_2019['Demand_Index'],'supply_point_index':df_demand_2019['Supply_Index'],'value':df_demand_2019['Demand_Satisfied']})
df_solution_supply_2019_FCS = pd.DataFrame({'year':2019,'data_type':'FCS','demand_point_index':'','supply_point_index':df_supply_2019['supply_point_index'],'value':df_supply_2019['existing_num_FCS']})
df_solution_supply_2019_SCS = pd.DataFrame({'year':2019,'data_type':'SCS','demand_point_index':'','supply_point_index':df_supply_2019['supply_point_index'],'value':df_supply_2019['existing_num_SCS']})
df_solution_supply_2020_FCS = pd.DataFrame({'year':2020,'data_type':'FCS','demand_point_index':'','supply_point_index':df_supply_2020['supply_point_index'],'value':df_supply_2020['existing_num_FCS']})
df_solution_supply_2020_SCS = pd.DataFrame({'year':2020,'data_type':'SCS','demand_point_index':'','supply_point_index':df_supply_2020['supply_point_index'],'value':df_supply_2020['existing_num_SCS']})
df_solution = pd.concat([df_solution_supply_2019_SCS,df_solution_supply_2019_FCS,df_solution_supply_2020_SCS,df_solution_supply_2020_FCS,df_solution_demand_2019,df_solution_demand_2020],axis=0)
df_solution
df_solution.reset_index(inplace = True)

#Round Values due to float rounding error, to satisfie constraint 5

def my_floor(a, precision=3):
    return np.round(a - 0.5 * 10**(-precision), precision)
df_solution['value'] = df_solution['value'].apply(my_floor)

df_solution.to_csv('./Output/Highscore_tests/solution.csv', index=False)